단어 → 수치화(DTM, Word2Vec) <br>
문서간 단어들의 차이를 계산? 유클리드, 코사인 유사도 등  <br><br>

유사도 = 두 객체가 얼마나 닮았는지를 나타내는 수치(측정)값 <br>
비유사도 = 두 객체가 얼마나 다른지를 나타내는 수치(측정)값 <br><br>

유클리디안 거리의 정의 <br>
n = 열의 수 = feature의 수 <br>
pk, qk = k 차원의 값 <br>
두 점사이의 거리가 가까우면 유사도 ↑ <br>

유클리디안 거리의 한계 <br>
(x="사랑", y = "돈") <br>
p1 (10, 0) <br>
p2 (1, 0) <br>
p3 (9, 5) <br>
p1과 p2 > p1과 p3: 더 가깝다 → 유사도가 더 높게 나온다  <br>
그럼 두 점사이의 **각도**도 따져보자 → **같은** 방향인지 <br><br>

코사인 유사도 <br>
두 벡터의 방향이 완전이 동일 : 1 <br>
90도 : 0 <br>
180 : -1 <br>

1에 가까우면 가까울수룩 유사도가 높다  <br>
분모 = 두 벡터의 길이 곱 <br>
분자 = 두 벡터의 내적(요소간 곱셈) <br> <br>

문서1 : 저는 파이썬 좋아요 <br>
문서2 : 저는 자바 좋아요 <br>
문서3 : 저는 자바 좋아요 저는 자바 좋아요 <br>

| <center>문서</center> |  <center>저는</center> |  <center>파이썬</center> |   <center>자바</center> |  <center>좋아요</center> |
|:--------|:--------:|--------:|--------:|--------:|
|**문서 1** | <center>1</center> | <center>1</center> | <center>0</center> | <center>1</center> |
|**문서 2** | <center>1</center> | <center>0</center> | <center>1</center> | <center>1</center> |
|**문서 3** | <center>2</center> | <center>0</center> | <center>2</center> | <center>2</center> |

문서1문서2 = (1+0+0+1) / 3 = 2/3 = 0.67  <br> 
문서1문서3 = (2+0+0+2) / 루트3*루트12 = 4/6 = 0.67  <br>
문서2문서3 = (2+0+2+2) / 6 = 6 / 6 = 1  <br> <br>

문서2와 문서3의 유사도가 제일 높다 → 서로 크기만 다르고 방향이 같은 벡터  <br> <br>

"토이스토리4" 영화와 가장 유사한 영화 검색  <br>
→ 줄거리, 코사인 유사도 기반  <br>
→ 각 영화에 대한 DTM 필요 <br> 
→ 유의어는 어떻게 처리할 것인가? <br>

In [2]:
import numpy as np
from numpy.linalg import norm # norm에도 종류가 여러가지가 있지만 보통 l2 norm

def cos_sim(x, y):
    return np.dot(x, y) / (norm(x)*norm(y))
    
doc1 = np.array([1, 1, 0, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc3, doc2))

0.6666666666666667
0.6666666666666667
1.0000000000000002


## 영화 줄거리를 이용한 유사도 분석 및 영화 추천
[[영화 데이터 출처]](https://www.kaggle.com/rounakbanik/the-movies-dataset)

In [3]:
import pandas as pd
data = pd.read_csv("./res/the-movies-dataset/movies_metadata.csv")
data.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
data.head()[["title", "overview"]]

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [18]:
data = data.head(20000) # 데이터량이 너무 많아서 2만개로만 실습

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 단순 코사인 유사도 기반 계산
tfidf = TfidfVectorizer()
tfidf # stop_words=None

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [20]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [21]:
data["overview"].isnull().value_counts() 

False    20000
Name: overview, dtype: int64

tfidf 작업 시 NaN이 있으면 error 발생

In [22]:
data["overview"] = data["overview"].fillna("") # NaN의 경우 ""로 대체
data["overview"].isnull().value_counts()

False    20000
Name: overview, dtype: int64

In [23]:
tfidf_mat = tfidf.fit_transform(data["overview"]) # 영화의 줄거리로 tfidf 생성
tfidf_mat.shape

(20000, 47487)

In [24]:
from sklearn.metrics.pairwise import linear_kernel # 코사인 유사도 함수
cos_sim = linear_kernel(tfidf_mat,tfidf_mat)

In [25]:
tfidf_mat

<20000x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 535909 stored elements in Compressed Sparse Row format>

In [26]:
tfidf_mat.toarray() # array로 확인하기 위해 변환, 각 행은 영화 1개에 대한 값 출력

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
cos_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [28]:
data.shape

(20000, 24)

In [29]:
idx = pd.Series(data.index, index=data["title"]).drop_duplicates() # 중복된 영화 제거
idx.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [30]:
print(idx["Rambo"])
print(cos_sim[12356]) #rambo 영화에 대한 코사인 유사도 
# 0값은 분자에 0이 있다는 의미로 둘 중에 0이 있다는 것으로 유사한 영화로 보기 어려움
# 0이 아닌 값은 유사도가 있다는 의미
print(type(cos_sim[12356]))

12356
[0.02696439 0.         0.02085587 ... 0.         0.         0.        ]
<class 'numpy.ndarray'>


In [33]:
# 코사인 유사도를 활용해 유사한 영화 10편 출력
def get_recommendations(title, cos_sim=cos_sim): #cos_sim 변수에 들어가는 값은 cos_sim 함수를 쓰겠다는 의미
    idx_title = idx[title] #12356=idx['Rambo']
    sim_score = list(enumerate(cos_sim[idx_title])) # 열거형으로 변환
    sim_score = sorted(sim_score, key=lambda x:x[1], reverse=True) 
    #튜플에서 유사도 값을 가지고 내림차순 정렬하기 위해 튜플 인덱스 1을 가지고 정렬하겠다는 의미
    mi = sim_score[1:11]
    res = [i[0] for i in mi]
    print(data["title"].iloc[res])

In [34]:
get_recommendations("Rambo") # 람보와 다른 영화 사이의 코사인 유사도 

2290                         First Blood
2291                           Rambo III
2289          Rambo: First Blood Part II
2693                   Universal Soldier
3832                          Billy Jack
9632     Strawberries in the Supermarket
1165                      Apocalypse Now
12252              Journey from the Fall
16638                     Little Soldier
10030                         The Search
Name: title, dtype: object


## POST 방식을 이용한 스크래핑
http://www.hanbit.co.kr/

In [43]:
import requests

login_info = {"m_id": "ID",
              "m_passwd": "비밀번호"}
url_login = "http://www.hanbit.co.kr/member/login_proc.php"

# POST 방식으로 서버에 연결
session = requests.session()
res = session.post(url_login, data=login_info)
res

<Response [200]>

In [55]:
url_mypage = "http://www.hanbit.co.kr/myhanbit/myhanbit.html"
res = session.get(url_mypage)
res

<Response [200]>

In [56]:
res.text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<!-- Google Tag Manager -->\r\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-W9D5PM3\');</script>\r\n<!-- End Google Tag Manager -->\r\n<meta charset="utf-8"/>\r\n<title>한빛출판네트워크</title>\r\n<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛출판네트워크"/>\r\n<meta property="og:description" content="출판사, IT전문서, 대학교재, 경제경영, 어린이/유아,

In [63]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, "html.parser")
# #container > div > div.sm_mymileage > dl.mileage_section1 > dd > span
soup.select_one(".mileage_section1 span").get_text()

'2,000'

In [61]:
# #container > div > div.sm_mymileage > dl.mileage_section2 > dd
soup.select_one("#container > div > div.sm_mymileage > dl.mileage_section2 > dd").get_text()

'0 원'